In [1]:
import heeps
from heeps.util.coord import mas2rms
from heeps.contrast.background import background
import astropy.units as u
from astropy.io import fits
import os.path
from copy import deepcopy
import matplotlib.pyplot as plt
%matplotlib inline

## General inputs

In [2]:
conf = dict(
    mode = 'CVC',
    file_pupil = 'pupil/ELT_fullM1.fits',
    file_phase = 'wavefront/cube_Cbasic_20201130_3600s_300ms_0piston_meters_scao_only_285.fits',
    file_point_err = 'wavefront/point_all_3600s_300ms.fits',
    add_phase = True,
    add_point_err = True,
    nframes = 220,
    cube_duration = 220*0.3,
)
# range of magnitudes
mags = np.arange(5, 12+1)
print('mags = %s'%mags)
# offsets:
# up to 0.1 lam/D, step = 1/100
# up to 1 lam/D, step = 1/10
xy = np.append(np.arange(100)/1000, np.arange(1,11)/10)
print('nb offsets = %s\noffsets = %s'%(xy.shape[0], xy))

mags = [ 5  6  7  8  9 10 11 12]
nb offsets = 110
offsets = [0.    0.001 0.002 0.003 0.004 0.005 0.006 0.007 0.008 0.009 0.01  0.011
 0.012 0.013 0.014 0.015 0.016 0.017 0.018 0.019 0.02  0.021 0.022 0.023
 0.024 0.025 0.026 0.027 0.028 0.029 0.03  0.031 0.032 0.033 0.034 0.035
 0.036 0.037 0.038 0.039 0.04  0.041 0.042 0.043 0.044 0.045 0.046 0.047
 0.048 0.049 0.05  0.051 0.052 0.053 0.054 0.055 0.056 0.057 0.058 0.059
 0.06  0.061 0.062 0.063 0.064 0.065 0.066 0.067 0.068 0.069 0.07  0.071
 0.072 0.073 0.074 0.075 0.076 0.077 0.078 0.079 0.08  0.081 0.082 0.083
 0.084 0.085 0.086 0.087 0.088 0.089 0.09  0.091 0.092 0.093 0.094 0.095
 0.096 0.097 0.098 0.099 0.1   0.2   0.3   0.4   0.5   0.6   0.7   0.8
 0.9   1.   ]


In [3]:
def create_psfs(conf, case, mags=None, verbose=False):
    # update dir_output
    conf.update(dir_output=case)
    # initialize wavefront
    conf = heeps.config.read_config(verbose=False, **conf)
    conf = heeps.config.update_config(saveconf=True, verbose=verbose, **conf)
    wf = heeps.pupil.pupil(savefits=True, verbose=verbose, **conf)
    # load errors
    phase_screens, amp_screens, tiptilts, misaligns = heeps.wavefront.load_errors(verbose=verbose, **conf)
    # off-axis PSF
    psf = heeps.wavefront.propagate_one(wf, onaxis=False, savefits=True, verbose=verbose, **conf)
    # linear tiptilt
    lamD = conf['lam']/conf['diam_nominal']*u.rad.to('mas')
    xy_rms = mas2rms(xy*lamD, conf['diam_nominal']) # converted to mas, then to rms
    tt = [(x,0) for x in xy_rms] + [(0,y) for y in xy_rms]
    # choose case
    print('\nCase = %s'%case)
    if 'linear_drift_only' in case:
        tiptilts = tt
        phase_screens = np.array([None]*conf['nframes'])
    elif 'scao_only' in case:
        tiptilts = np.array([None]*conf['nframes'])
    elif 'scao+linear_drift' in case:
        tiptilts = tt
    elif 'scao+random_drift' in case:
        pass
    # on-axis PSFs
    psfs = heeps.wavefront.propagate_cube(wf, phase_screens=phase_screens, \
        amp_screens=amp_screens, tiptilts=tiptilts, misaligns=misaligns, onaxis=True, \
        savefits=True, verbose=True, **conf)
    # add background
    if mags is not None:
        print('mag', end='')
        for conf['mag'] in mags:
            print(', %s'%conf['mag'], end='')
            psf_ON, psf_OFF = background(deepcopy(psfs), deepcopy(psf), **conf)
            fits.writeto(os.path.join(conf['dir_output'], 'onaxis_PSF_%s_%s_mag%s.fits'\
                %(conf['band'], conf['mode'], conf['mag'])), np.float32(psf_ON), overwrite=True)
            fits.writeto(os.path.join(conf['dir_output'], 'offaxis_PSF_%s_%s_mag%s.fits'\
                %(conf['band'], conf['mode'], conf['mag'])), np.float32(psf_OFF), overwrite=True)
        print('')

## L-band CVC

In [4]:
conf.update(
    band = 'L',
    file_lyot_stop = 'pupil/ls_LM_CVC_0.0291_0.08_0.0317_285.fits',
)
create_psfs(conf, 'qacits_L/linear_drift_only', verbose=True)
create_psfs(conf, 'qacits_L/scao_only', mags=mags)
create_psfs(conf, 'qacits_L/scao+linear_drift', mags=mags)
create_psfs(conf, 'qacits_L/scao+random_drift', mags=mags)

Simulation config: band=L, mode=CVC
‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾
   loading vortex back-propagation params
   vc_charge=2, ngrid=1024, beam_ratio=0.2568
   npupil=285, pscale=5.47 mas, lam=3.8112E-06 m
   hfov=1.1 arcsec (-> ndet=403, 51.74 lam/D)

Load pupil from 'ELT_fullM1.fits'
Load phase screens from 'cube_Cbasic_20201130_3600s_300ms_0piston_meters_scao_only_285.fits'
   nscreens=220 (nframes=220, nstep=1)
Load pointing errors from 'point_all_3600s_300ms.fits'
   nscreens=220
Create single off-axis PSF
   apply lyot stop from 'ls_LM_CVC_0.0291_0.08_0.0317_285.fits'
   extract PSF on the detector: ndet=403

Case = qacits_L/linear_drift_only
Create on-axis PSF cube
   2021-05-17 17:08:01, e2e simulation using 56 cores
   expected time to complete: 6.96 seconds
   2021-05-17 17:10:05, completed in 125.48 seconds

Case = qacits_L/scao_only
Create on-axis PSF cube
   2021-05-17 17:10:09, e2e simulation using 56 cores
   expected time to complete: 4.71 seconds
   2021-05-17 17:12:15, completed in

## N2-band CVC

In [5]:
conf.update(
    band = 'N2',
    file_lyot_stop = 'pupil/ls_N_CVC_0.0356_0.08_0.0383_119.fits',
)
create_psfs(conf, 'qacits_N2/linear_drift_only', verbose=True)
create_psfs(conf, 'qacits_N2/scao_only', mags=mags)
create_psfs(conf, 'qacits_N2/scao+linear_drift', mags=mags)
create_psfs(conf, 'qacits_N2/scao+random_drift', mags=mags)

Simulation config: band=N2, mode=CVC
‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾
   loading vortex back-propagation params
   vc_charge=2, ngrid=1024, beam_ratio=0.1072
   npupil=119, pscale=6.79 mas, lam=1.1330E-05 m
   hfov=1.1 arcsec (-> ndet=325, 17.42 lam/D)

Load pupil from 'ELT_fullM1.fits'
Load phase screens from 'cube_Cbasic_20201130_3600s_300ms_0piston_meters_scao_only_285.fits'
   nscreens=220 (nframes=220, nstep=1)
Load pointing errors from 'point_all_3600s_300ms.fits'
   nscreens=220
Create single off-axis PSF
   apply lyot stop from 'ls_N_CVC_0.0356_0.08_0.0383_119.fits'
   extract PSF on the detector: ndet=325

Case = qacits_N2/linear_drift_only
Create on-axis PSF cube
   2021-05-17 17:17:46, e2e simulation using 56 cores
   expected time to complete: 6.17 seconds
   2021-05-17 17:19:52, completed in 126.86 seconds

Case = qacits_N2/scao_only
Create on-axis PSF cube
   2021-05-17 17:19:55, e2e simulation using 56 cores
   expected time to complete: 6.09 seconds
   2021-05-17 17:21:58, completed 